In [5]:
import numpy as np
import pandas as pd
import torch

import new_module.losses as lossbuilder
from new_module.utils.robertacustom import RobertaCustomForSequenceClassification
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel, pipeline,AutoModelForCausalLM

In [2]:
import os

os.environ['HF_HOME']='/shared/s3/lab07/hyeryung/hf_cache'
os.environ['HF_DATASETS_CACHE']='/shared/s3/lab07/hyeryung/hf_cache'
os.environ['TRANSFORMERS_CACHE']='/shared/s3/lab07/hyeryung/hf_cache'

In [6]:
model = AutoModelForCausalLM.from_pretrained('gpt2-large',cache_dir='/shared/s3/lab07/hyeryung/hf_cache')

In [34]:
tokenizer = AutoTokenizer.from_pretrained('gpt2-large',cache_dir='/shared/s3/lab07/hyeryung/hf_cache')

In [23]:
prompts=["Once upon a time",
"The book",
"The chicken",
"The city",
"The country",
"The horse",
"The lake",
"The last time",
"The movie",
"The painting",
"The pizza",
"The potato",
"The president of the country",
"The road",
"The year is 1910"]

In [47]:
gens_data_all=[]
for seq_len in [12,20,50]:
    pipe = pipeline(task='text-generation',
                model=model,
                tokenizer=tokenizer,
                device='cuda:0',
                do_sample=True,
                top_p=1,
                max_length=seq_len,
                num_return_sequences=20,
                pad_token_id=50256)
    gens_all=[]
    for prompt in prompts:
        gens = pipe(prompt)
        for gen in gens:
            gen['generated_text']=gen['generated_text'][len(prompt):]
            gen['text']=gen['generated_text']
            del gen['generated_text']
        gens_all.append(gens)
    gens_data=pd.DataFrame({'prompt': [{'text':x} for x in prompts], 'generations': gens_all, 'seq_lengths': [seq_len]*len(prompts)})
    gens_data_all.append(gens_data)
gens_data_all=pd.concat(gens_data_all)

/home/s3/hyeryung/miniconda3/envs/loc-edit/lib/python3.11/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [49]:
gens_data_all.to_json('new_module/data/sentiment/dev_set.jsonl', orient='records', lines=True)

In [45]:
gens_data

,prompt,generations
0,{'text': 'Once upon a time'},"[{'text': ', many people who thought the world..."
1,{'text': 'The book'},"[{'text': ' of Leviticus 18, called Deuteronom..."
2,{'text': 'The chicken'},"[{'text': ' is very lean, it's about 20 pounds..."
3,{'text': 'The city'},"[{'text': ' had already started a program to ""..."
4,{'text': 'The country'},[{'text': ''s political leaders seemed to be o...
5,{'text': 'The horse'},"[{'text': ' was a good example,"" Covington sai..."
6,{'text': 'The lake'},[{'text': ' of the world is only water; but th...
7,{'text': 'The last time'},"[{'text': ' I remember me, I had black hair an..."
8,{'text': 'The movie'},[{'text': ' has also attracted criticism. In o...
9,{'text': 'The painting'},[{'text': ' is being displayed on display at t...


In [35]:
tokenizer = AutoTokenizer.from_pretrained('/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-2/step_2600_best_checkpoint/')

In [3]:
## read original data
gpt2_outputs=pd.read_json('new_module/data/toxicity-avoidance/testset_gpt2_2500.jsonl', lines=True)

In [19]:
config={'build_loss_dict': {"coeff_steps": 200, "coeff_pattern": "constant", "loss_type": "xentropy", "length_normalize": False, "AR_temperature": 1.0, "AR_top_k": 0, "AR_top_p": 0.96, "max_output_length": 20},
       }
class dummyArgs:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

build_loss_args = dummyArgs(**config["build_loss_dict"])

In [21]:
ckpts = [### clsf-embed-share
'/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-2/step_2600_best_checkpoint/',
### em-embed-share
'/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-energy-training/step_2800_best_checkpoint/',
### clsf
'/shared/s3/lab07/hyeryung/loc_edit/models_re/roberta-base-jigsaw-toxicity-classifier/step_500_best_checkpoint/',
### em
'/shared/s3/lab07/hyeryung/loc_edit/models_re/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint/'
]

In [23]:
lossfns=[]
for ckpt in ckpts:
    if 'with-gpt2-large-embeds' in ckpt:
        model=RobertaCustomForSequenceClassification.from_pretrained(ckpt)
    else:
        model=AutoModelForSequenceClassification.from_pretrained(ckpt)
    model.eval()
    tokenizer=AutoTokenizer.from_pretrained(ckpt)
    
    lossfn = \
        lossbuilder.build_loss(
            "classification_no_prefix_logprobloss",
            model,
            tokenizer,
            build_loss_args,
        )
    lossfns.append(lossfn)

50265
50265


In [39]:
source_texts = []
AR_predictions = []

for i, row in gpt2_outputs.iterrows():
    
    source_tokens = row.prompt['tokens']
    source_text = tokenizer.decode(source_tokens)
    
    curr_predictions = []
    for j in range(10):
        predicted_tokens = row.generations[j]['tokens']
        AR_prediction = tokenizer.decode(predicted_tokens)
        lossvals = np.array([])
        with torch.no_grad():
            for lossfn in lossfns:
                lossvalue = lossfn.compute_gold_loss(
                    source_text, AR_prediction,
                    label_id=0,
                )
                lossvals = np.append(lossvals, lossvalue.squeeze().item())
        if (lossvals > -np.log(0.75)).sum() >= 1:
            curr_predictions.append({'text':AR_prediction})
    source_texts.append({'text':source_text})
    AR_predictions.append(curr_predictions)
        

In [32]:
## select examples that are problematic
pd.DataFrame({'prompt': source_texts, 'generations': AR_predictions})


,prompt,generations
0,{'text': 'PP June ever value already hasn left...,[{'text': '- Slip hockey for'}]
1,{'text': ' are denies interestingweight murder...,[]
2,{'text': ' workingA knows left counterparts wo...,[{'text': ' front risk family system family se...
3,{'text': ' learn There shortas invited 25A loc...,[]
4,{'text': ' implementation brother in win calle...,[{'text': ' July workscent 8 35ucraft family 7...
...,...,...
245,{'text': ' land If Cabinet added news rifles F...,[{'text': ' BB in 8 principal forado in news L...
246,{'text': ' journalism seem 15 7 treaties steps...,[]
247,"{'text': ' areback leave towncount"",back t Act...",[{'text': ' abuse for'}]
248,{'text': 'inations Tommy called Dr claimed fam...,[{'text': 'ization As Liverpool hour said ensu...
